In [1]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [2]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
            result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [4]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [5]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [12]:
X,x,Y,y = np.array(x_train),np.array(x_test),np.array(y_train),np.array(y_test)

In [7]:
glob.glob("C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav")

['C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-01-01-01-01-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-01-01-01-02-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-01-01-02-01-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-01-01-02-02-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-02-01-01-01-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-02-01-01-02-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-02-01-02-01-01.wav',
 'C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_01\\03-01-02-01-02-02-01.wav',
 'C:\\Us

In [8]:
24*60

1440

In [9]:
len(glob.glob("C:\\Users\\user\\Downloads\\speech-emotion\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"))

1440

In [10]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [13]:
print(X.shape)
print(Y.shape)
print(x.shape)
print(y.shape)

(576, 180)
(576,)
(192, 180)
(192,)


In [14]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [15]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,),
                    learning_rate='adaptive', max_iter=500)

In [16]:
model.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(300,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [17]:
y_pred=model.predict(x_test)

In [18]:
y_pred

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'disgust',
       'happy', 'disgust', 'happy', 'happy', 'disgust', 'fearful',
       'happy', 'disgust', 'happy', 'calm', 'disgust', 'happy', 'disgust',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy',
       'fearful', 'happy', 'fearful', 'fearful', 'happy', 'disgust',
       'happy', 'fearful', 'happy', 'calm', 'calm', 'fearful', 'calm',
       'disgust', 'happy', 'calm', 'calm', 'calm', 'disgust', 'disgust',
       'disgust', 'happy', 'calm', 'happy', 'fearful', 'fearful',
       'fearful', 'fearful', 'happy', 'fearful', 'disgust', 'happy',
       'calm', 'calm', 'disgust', 'calm', 'disgust', 'calm', 'disgust',
       'calm', 'calm', 'disgust', 'disgust', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'fearful', 'disgust', 'fearful',
       'happy', 'calm', 'fearful', 'disgust', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'disgust', 'fearful', 'disgust',
       'fearful', 'disgust'

In [19]:
accuracy_score(y_test,y_pred)

0.7760416666666666

In [20]:
Y

array(['disgust', 'fearful', 'calm', 'calm', 'fearful', 'fearful',
       'disgust', 'calm', 'calm', 'happy', 'calm', 'disgust', 'fearful',
       'disgust', 'fearful', 'disgust', 'happy', 'disgust', 'fearful',
       'happy', 'happy', 'calm', 'fearful', 'disgust', 'calm', 'fearful',
       'disgust', 'fearful', 'calm', 'calm', 'happy', 'fearful', 'calm',
       'happy', 'fearful', 'calm', 'fearful', 'disgust', 'calm',
       'disgust', 'happy', 'fearful', 'happy', 'fearful', 'calm', 'happy',
       'happy', 'calm', 'fearful', 'disgust', 'disgust', 'disgust',
       'calm', 'happy', 'calm', 'calm', 'calm', 'disgust', 'happy',
       'disgust', 'happy', 'fearful', 'happy', 'disgust', 'happy',
       'fearful', 'calm', 'happy', 'calm', 'happy', 'disgust', 'fearful',
       'calm', 'calm', 'happy', 'fearful', 'calm', 'calm', 'happy',
       'fearful', 'happy', 'calm', 'happy', 'fearful', 'happy', 'disgust',
       'disgust', 'fearful', 'fearful', 'disgust', 'disgust', 'fearful',
       'c